# جانوروں کے ماہر نظام کا نفاذ

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) سے ایک مثال۔

اس نمونے میں، ہم ایک سادہ علم پر مبنی نظام نافذ کریں گے جو کچھ جسمانی خصوصیات کی بنیاد پر جانور کا تعین کرے گا۔ اس نظام کو درج ذیل AND-OR درخت کے ذریعے ظاہر کیا جا سکتا ہے (یہ پورے درخت کا ایک حصہ ہے، ہم آسانی سے مزید قواعد شامل کر سکتے ہیں):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.ur.png)


## ہمارا اپنا ماہر نظام شیل بیکورڈ انفرنس کے ساتھ

آئیے پیداوار کے اصولوں پر مبنی علم کی نمائندگی کے لیے ایک سادہ زبان کی تعریف کرنے کی کوشش کریں۔ ہم قواعد کی تعریف کے لیے Python کلاسز کو بطور کلیدی الفاظ استعمال کریں گے۔ بنیادی طور پر تین قسم کی کلاسز ہوں گی:
* `Ask` ایک سوال کی نمائندگی کرتا ہے جو صارف سے پوچھا جانا ضروری ہے۔ اس میں ممکنہ جوابات کا سیٹ شامل ہوتا ہے۔
* `If` ایک اصول کی نمائندگی کرتا ہے، اور یہ صرف اصول کے مواد کو ذخیرہ کرنے کے لیے ایک نحوی سہولت ہے۔
* `AND`/`OR` درخت کی AND/OR شاخوں کی نمائندگی کرنے کے لیے کلاسز ہیں۔ یہ صرف دلائل کی فہرست کو اندر ذخیرہ کرتے ہیں۔ کوڈ کو آسان بنانے کے لیے، تمام فعالیت والدین کلاس `Content` میں بیان کی گئی ہے۔


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

ہمارے نظام میں، کام کرنے والی میموری میں **حقائق** کی فہرست **صفات-قدر جوڑوں** کے طور پر شامل ہوگی۔ علم کا ذخیرہ ایک بڑا لغت کے طور پر بیان کیا جا سکتا ہے جو اعمال (نئے حقائق جو کام کرنے والی میموری میں شامل کیے جانے چاہئیں) کو شرائط کے ساتھ نقشہ کرتا ہے، جو AND-OR اظہارات کے طور پر ظاہر کیے جاتے ہیں۔ نیز، کچھ حقائق کو `پوچھا` جا سکتا ہے۔


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

پچھلی استدلالی عمل کو انجام دینے کے لیے، ہم `Knowledgebase` کلاس کو ڈیفائن کریں گے۔ یہ درج ذیل پر مشتمل ہوگی:
* کام کرنے والی `memory` - ایک ڈکشنری جو خصوصیات کو ان کی قدروں کے ساتھ جوڑتی ہے
* Knowledgebase کے `rules` - اوپر بیان کردہ فارمیٹ میں

دو اہم طریقے یہ ہیں:
* `get` کسی خصوصیت کی قدر حاصل کرنے کے لیے، اگر ضروری ہو تو استدلالی عمل انجام دے گا۔ مثال کے طور پر، `get('color')` رنگ کے سلاٹ کی قدر حاصل کرے گا (یہ اگر ضروری ہو تو پوچھے گا، اور بعد میں استعمال کے لیے کام کرنے والی میموری میں قدر محفوظ کرے گا)۔ اگر ہم `get('color:blue')` پوچھیں، تو یہ رنگ کے بارے میں پوچھے گا، اور پھر رنگ کے مطابق `y`/`n` قدر واپس کرے گا۔
* `eval` اصل استدلالی عمل انجام دیتا ہے، یعنی AND/OR درخت کو عبور کرتا ہے، ذیلی مقاصد کا جائزہ لیتا ہے، وغیرہ۔


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

اب آئیے ہم اپنے جانوروں کے علم کے ذخیرے کو متعین کریں اور مشاورت انجام دیں۔ نوٹ کریں کہ یہ کال آپ سے سوالات پوچھے گی۔ آپ `y`/`n` کے ذریعے ہاں-نہیں کے سوالات کے جواب دے سکتے ہیں، یا طویل متعدد انتخابی جوابات والے سوالات کے لیے نمبر (0..N) بتا سکتے ہیں۔


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## PyKnow کا استعمال کرتے ہوئے فارورڈ انفرنس

اگلی مثال میں، ہم علم کی نمائندگی کے لیے ایک لائبریری [PyKnow](https://github.com/buguroo/pyknow/) کا استعمال کرتے ہوئے فارورڈ انفرنس کو نافذ کرنے کی کوشش کریں گے۔ **PyKnow** ایک لائبریری ہے جو Python میں فارورڈ انفرنس سسٹمز بنانے کے لیے استعمال ہوتی ہے، اور یہ کلاسیکی پرانے سسٹم [CLIPS](http://www.clipsrules.net/index.html) سے مشابہت رکھنے کے لیے ڈیزائن کی گئی ہے۔

ہم خود بھی فارورڈ چیننگ کو بغیر کسی بڑی مشکل کے نافذ کر سکتے تھے، لیکن سادہ طریقے عام طور پر زیادہ مؤثر نہیں ہوتے۔ زیادہ مؤثر اصولوں کے ملاپ کے لیے ایک خاص الگورتھم [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) استعمال کیا جاتا ہے۔


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

ہم اپنے نظام کو ایک کلاس کے طور پر تعریف کریں گے جو `KnowledgeEngine` کو سب کلاس کرے گی۔ ہر اصول کو ایک الگ فنکشن کے ذریعے تعریف کیا جاتا ہے جس میں `@Rule` تشریح ہوتی ہے، جو یہ بتاتی ہے کہ اصول کب فعال ہونا چاہیے۔ اصول کے اندر، ہم `declare` فنکشن کا استعمال کرتے ہوئے نئے حقائق شامل کر سکتے ہیں، اور ان حقائق کو شامل کرنے سے مزید اصولوں کو فارورڈ انفرنس انجن کے ذریعے فعال کیا جائے گا۔


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

ایک بار جب ہم نے نالج بیس کی تعریف کر لی، تو ہم اپنی ورکنگ میموری کو کچھ ابتدائی حقائق کے ساتھ بھر دیتے ہیں، اور پھر `run()` میتھڈ کو کال کرتے ہیں تاکہ استنتاج انجام دیا جا سکے۔ آپ نتیجے کے طور پر دیکھ سکتے ہیں کہ نئے استنتاج شدہ حقائق ورکنگ میموری میں شامل کیے جاتے ہیں، بشمول جانور کے بارے میں آخری حقیقت (اگر ہم نے تمام ابتدائی حقائق کو درست طریقے سے ترتیب دیا ہو)۔


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
